In [1]:
import pandas as pd
import csv
import os
import numpy as np
import datetime
from xlsxwriter.workbook import Workbook
from xlsxwriter import Workbook
import time

In [2]:
Working_dir = os.getcwd()
Data_dir = os.path.join(Working_dir, "Raw data")

In [3]:
node_df = pd.read_csv( os.path.join( Data_dir,"May2018_idle.csv"), delimiter=";", header=0 )

In [4]:
node_df.head()

,nodename,tempo,timestamp_measure,sys_power,cpu_power,mem_power,fan1a,fan1b,fan2a,fan2b,...,sys_util,cpu_util,mem_util,io_util,amb_temp,cpu1_temp,cpu2_temp,exh_temp,sysairflow,dcenergy
0,cresco6x001,Tue 15 May 16:05:48 CEST 2018,1526393148,150,110,13,3584,3712,4736,4992,...,0,0,0,0,19,45,40,38,13,"260,19502"
1,cresco6x002,Tue 15 May 16:05:58 CEST 2018,1526393148,150,110,12,3584,3712,4736,4992,...,0,0,0,0,18,45,40,38,13,"251,28987"
2,cresco6x003,Tue 15 May 16:06:07 CEST 2018,1526393148,150,110,12,3584,3712,4736,4992,...,0,0,0,0,20,47,39,40,13,"253,87433"
3,cresco6x004,Tue 15 May 16:06:16 CEST 2018,1526393148,150,110,10,3584,3712,4736,4992,...,0,0,0,0,17,44,37,37,13,"232,74909"
4,cresco6x005,Tue 15 May 16:06:26 CEST 2018,1526393148,150,110,13,3712,3712,4736,4992,...,0,0,0,0,19,43,37,38,13,"243,38527"


In [5]:
node_df.nodename.unique()

array(['cresco6x001', 'cresco6x002', 'cresco6x003', 'cresco6x004',
       'cresco6x005', 'cresco6x006', 'cresco6x007', 'cresco6x008',
       'cresco6x009', 'cresco6x010', 'cresco6x011', 'cresco6x012',
       'cresco6x013', 'cresco6x014', 'cresco6x015', 'cresco6x016',
       'cresco6x017', 'cresco6x018', 'cresco6x019', 'cresco6x020',
       'cresco6x021', 'cresco6x022', 'cresco6x023', 'cresco6x024',
       'cresco6x025', 'cresco6x026', 'cresco6x027', 'cresco6x028',
       'cresco6x029', 'cresco6x030', 'cresco6x031', 'cresco6x032',
       'cresco6x033', 'cresco6x034', 'cresco6x035', 'cresco6x036',
       'cresco6x037', 'cresco6x038', 'cresco6x039', 'cresco6x040',
       'cresco6x041', 'cresco6x042', 'cresco6x043', 'cresco6x044',
       'cresco6x045', 'cresco6x046', 'cresco6x047', 'cresco6x048',
       'cresco6x049', 'cresco6x050', 'cresco6x051', 'cresco6x052',
       'cresco6x053', 'cresco6x054', 'cresco6x055', 'cresco6x056',
       'cresco6x057', 'cresco6x058', 'cresco6x059', 'cresco6x0

In [6]:
print(node_df.columns.values)

['nodename' 'tempo' 'timestamp_measure' 'sys_power' 'cpu_power'
 'mem_power' 'fan1a' 'fan1b' 'fan2a' 'fan2b' 'fan3a' 'fan3b' 'fan4a'
 'fan4b' 'fan5a' 'fan5b' 'sys_util' 'cpu_util' 'mem_util' 'io_util'
 'amb_temp' 'cpu1_temp' 'cpu2_temp' 'exh_temp' 'sysairflow' 'dcenergy']


In [7]:
node_df.dcenergy = node_df.dcenergy.apply(lambda x: float(x.replace(",", ".")) )

In [8]:
nodes_stats = pd.DataFrame(columns = list(node_df.nodename.unique()))

In [9]:
nodes_stats.loc["total_energy", :] = 0

-----
## Energy used by node
- for system, CPU and memory, rectangle formula of integration is applied
- dcenergy is simply summed

In [10]:
node_df.loc[node_df.shape[0]-1, "tempo"]

'Thu 31 May 23:52:58 CEST 2018'

In [11]:
node_df.tail()

,nodename,tempo,timestamp_measure,sys_power,cpu_power,mem_power,fan1a,fan1b,fan2a,fan2b,...,sys_util,cpu_util,mem_util,io_util,amb_temp,cpu1_temp,cpu2_temp,exh_temp,sysairflow,dcenergy
242806,cresco6x212,Thu 31 May 23:52:55 CEST 2018,1527803455,180,140,12,4736,4736,6144,6528,...,0,0,0,0,18,54,48,42,13,337.38243
242807,cresco6x213,Thu 31 May 23:52:56 CEST 2018,1527803455,190,140,15,4352,4736,6016,6144,...,0,0,0,0,19,54,48,38,13,338.57096
242808,cresco6x214,Thu 31 May 23:52:57 CEST 2018,1527803455,180,140,13,4608,3968,6016,5120,...,0,0,0,0,18,54,49,42,13,323.52585
242809,cresco6x215,Thu 31 May 23:52:57 CEST 2018,1527803455,180,140,13,4224,4480,5504,5760,...,0,0,0,0,18,52,47,36,13,314.07424
242810,cresco6x216,Thu 31 May 23:52:58 CEST 2018,1527803455,150,110,14,4352,4736,6016,6144,...,0,0,0,0,18,45,42,37,13,292.91054


In [12]:
# column = nodes_stats.columns.values[0]

In [13]:
# one_row_num = node_df[node_df.nodename == column].shape[0]
# node_df[node_df.nodename == column]["sys_power"][:-1].shape[0]
# one_node_df["timestamp_measure"][:-1].shape[0]

In [17]:
for col in nodes_stats.columns.values:
    
#     one_node_df = node_df[node_df.nodename == col].copy()
#     row_num = one_node_df.shape[0]
    
#     sys_energy = one_node_df["sys_power"][:-1]
#     cpu_energy = one_node_df["cpu_power"][:-1]
#     mem_energy = one_node_df["mem_power"][:-1]

#     time_difference = one_node_df["timestamp_measure"][1:] -\
#     one_node_df["timestamp_measure"][:-1] 
#     print(time_difference.shape[0])
# #     sys_energy = one_node_df.loc[:-1, "sys_power"]
# #     cpu_energy = one_node_df.loc[:-1, "cpu_power"]
# #     mem_energy = one_node_df.loc[:-1, "mem_power"]

# #     time_difference = one_node_df.loc[1:, "timestamp_measure"] -\
# #     one_node_df.loc[:-1, "timestamp_measure"] 
    
#     nodes_stats.loc["sys_power", col] = np.dot(sys_energy.values, time_difference)
#     nodes_stats.loc["cpu_power", col] = np.dot(cpu_energy.values, time_difference)
#     nodes_stats.loc["mem_power", col] = np.dot(mem_energy.values, time_difference)
    
    nodes_stats.loc["total_energy", col] = node_df[node_df.nodename == col]["dcenergy"].max() -\
    node_df[node_df.nodename == col]["dcenergy"].min()

In [18]:
nodes_stats

,cresco6x001,cresco6x002,cresco6x003,cresco6x004,cresco6x005,cresco6x006,cresco6x007,cresco6x008,cresco6x009,cresco6x010,...,cresco6x207,cresco6x208,cresco6x209,cresco6x210,cresco6x211,cresco6x212,cresco6x213,cresco6x214,cresco6x215,cresco6x216
total_energy,65.47448,75.14104,73.64083,74.99218,302.25762,75.18743,72.36876,74.06825,76.06666,75.41751,...,71.93825,73.77976,73.08272,71.17143,69.58181,72.13702,74.33528,71.54143,69.20202,56.63354


In [16]:
# excel_writer_may = pd.ExcelWriter( os.path.join( Data_dir, "LSF_Cresco6_cleaned.xlsx"))
# lsf_df_cleaned_v1.to_excel(excel_writer_lsf_cleaned)
# excel_writer_lsf_cleaned.save()

## Resume
- agreed: for columns *_util data is not available 
- 216 different nodes